In [ ]:
!pip install transformers datasets accelerate scikit-learn matplotlib
!pip install datasets==3.6.0


In [ ]:
import GQA_utils
import torch
from torch import nn
import torch.nn.functional as F
from datasets import load_dataset
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import numpy as np
from collections import defaultdict
import math
import torch.nn.init as init
from tqdm.auto import tqdm # Import tqdm for progress bar
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling
from torch.utils.data import DataLoader


# GQA test for CKA relations with Mean Pooling

## Set Model For GQA test

In [ ]:
model_name = "gpt2"
model, tokenizer = GQA_utils.get_model(model_name)
GQA_utils.extract_cka(model, tokenizer)
torch.save(GQA_utils.origin_cache, "init.pt")
model

In [ ]:
embed_dim = model.config.n_embd
for i in range(len(model.transformer.h)):
  gqa_layer = GQA_utils.GQAAttention(model.transformer.h[i].attn, K=4, layer_idx=i, CKA_data=True)
  model.transformer.h[i].attn = gqa_layer


In [ ]:
model

## Uptraining

In [ ]:
matrix_evolution = {}

GQA_log = GQA_utils.train_model(model, tokenizer, matrix_evolution, CKA_mode=True)

In [ ]:
torch.save(GQA_utils.cka_data, "mean_pool.pt")

# GQA test for projection matrices movement with Random Initialization

## Set Model For GQA test

In [ ]:
model_name = "gpt2"
model, tokenizer = GQA_utils.get_model(model_name)
model

In [ ]:
embed_dim = model.config.n_embd
for i in range(len(model.transformer.h)):
  gqa_layer = GQA_utils.GQAAttention(model.transformer.h[i].attn, K=4, mean_pool=False, CKA_data=True)
  model.transformer.h[i].attn = gqa_layer


## Uptraining

In [ ]:
matrix_evolution = {}

GQA_rand_log = GQA_utils.train_model(model, tokenizer, matrix_evolution, CKA_mode=True)

In [ ]:
torch.save(GQA_utils.cka_data, "random.pt")